<a href="https://colab.research.google.com/github/tamnguyen135/CPSC340-MachineLearning/blob/main/Project/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis with Deep Learning using BERT

### Prerequisites

- Intermediate-level knowledge of Python 3 (NumPy and Pandas preferably, but not required)
- Exposure to PyTorch usage
- Basic understanding of Deep Learning and Language Models (BERT specifically)

### Project Outline

**Task 1**: Introduction (this section)

**Task 2**: Exploratory Data Analysis and Preprocessing

**Task 3**: Training/Validation Split

**Task 4**: Loading Tokenizer and Encoding our Data

**Task 5**: Setting up BERT Pretrained Model

**Task 6**: Creating Data Loaders

**Task 7**: Setting Up Optimizer and Scheduler

**Task 8**: Defining our Performance Metrics

**Task 9**: Creating our Training Loop

**Task 10**: Loading and Evaluating our Model

## Task 1: Introduction

### What is BERT

BERT is a large-scale transformer-based Language Model that can be finetuned for a variety of tasks.

For more information, the original paper can be found [here](https://arxiv.org/abs/1810.04805). 

[HuggingFace documentation](https://huggingface.co/transformers/model_doc/bert.html)

[Bert documentation](https://characters.fandom.com/wiki/Bert_(Sesame_Street) ;)

<img src="Images/BERT_diagrams.pdf" width="1000">

## Task 2: Exploratory Data Analysis and Preprocessing

We will use the SMILE Twitter dataset.

_Wang, Bo; Tsakalidis, Adam; Liakata, Maria; Zubiaga, Arkaitz; Procter, Rob; Jensen, Eric (2016): SMILE Twitter Emotion dataset. figshare. Dataset. https://doi.org/10.6084/m9.figshare.3187909.v2_

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
cd drive/MyDrive/Colab\ Notebooks

[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks'
/content/drive/MyDrive/Colab Notebooks


In [9]:
cd Sentimental\ Analysis

/content/drive/MyDrive/Colab Notebooks/Sentimental Analysis


In [10]:
cd Project/

/content/drive/MyDrive/Colab Notebooks/Sentimental Analysis/Project


This is where it actually starts

In [11]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [12]:
df = pd.read_csv('Data/smile-annotations-final.csv', names = ['id', 'text', 'category'])
df.set_index('id', inplace = True)

In [13]:
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [14]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

In [15]:
df = df[df.category != 'nocode']

In [16]:
df = df[-df.category.str.contains('\|')]

In [17]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [18]:
possible_labels = df.category.unique()
possible_labels

array(['happy', 'not-relevant', 'angry', 'disgust', 'sad', 'surprise'],
      dtype=object)

In [19]:
label_dict = dict(zip(possible_labels, range(6)))
label_dict

{'angry': 2,
 'disgust': 3,
 'happy': 0,
 'not-relevant': 1,
 'sad': 4,
 'surprise': 5}

In [20]:
df.head()

,text,category
id,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy


In [21]:
df['label'] = df.category.map(label_dict)

In [22]:
df.columns

Index(['text', 'category', 'label'], dtype='object')

In [23]:
df.head(20)

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0
614499696015503361,Lucky @FitzMuseum_UK! Good luck @MirandaStearn...,happy,0
613601881441570816,Yr 9 art students are off to the @britishmuseu...,happy,0
613696526297210880,@RAMMuseum Please vote for us as @sainsbury #s...,not-relevant,1
610746718641102848,#AskTheGallery Have you got plans to privatise...,not-relevant,1


## Task 3: Training/Validation Split

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size = 0.15, 
                                                  random_state = 17,
                                                  stratify = df.label.values)

In [26]:
df['data_type'] = ['not_set'] * df.shape[0]

In [27]:
df.loc[X_train, ['data_type']] = 'train'
df.loc[X_val, ['data_type']] = 'val'

In [28]:
df.groupby(['category', 'label', 'data_type']).count()

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

## Task 4: Loading Tokenizer and Encoding our Data

In [29]:
!pip install transformers
!pip install --upgrade tensorflow

     |████████████████████████████████| 1.3MB 7.7MB/s 
     |████████████████████████████████| 890kB 22.9MB/s 
     |████████████████████████████████| 1.1MB 31.4MB/s 
     |████████████████████████████████| 2.9MB 43.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=b12207d627fca78a2ca136eb3dd57ad2c663781008a399993bbb9a5ba4668bf6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 320.4MB 49kB/s 
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [30]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [31]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
)

In [32]:
encoded_data_train = tokenizer. batch_encode_plus(
    df[df.data_type == 'train'].text.values,
    add_special_tokens = True,
    # sentences have different length, 
    # but we want output tensors to be of the same length
    return_attention_mask = True,
    pad_to_max_length = True,
    # a tweet has at most 256 words
    max_length = 256,
    return_tensors = 'pt'
)
encoded_data_val = tokenizer. batch_encode_plus(
    df[df.data_type == 'val'].text.values,
    add_special_tokens = True,
    # sentences have different length, 
    # but we want output tensors to be of the same length
    return_attention_mask = True,
    pad_to_max_length = True,
    # a tweet has at most 256 words
    max_length = 256,
    return_tensors = 'pt'
)
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type == 'train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type == 'val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [33]:
len(dataset_train)

1258

In [34]:
len(dataset_val)

223

## Task 5: Setting up BERT Pretrained Model

In [35]:
from transformers import BertForSequenceClassification

In [36]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = len(possible_labels),
    output_attentions = False,
    output_hidden_states = False
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Task 6: Creating Data Loaders

In [37]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [38]:
batch_size = 4

dataloader_train = DataLoader(dataset_train, 
                              sampler = RandomSampler(dataset_train), 
                              batch_size = batch_size)


dataloader_val = DataLoader(dataset_val, 
                              sampler = SequentialSampler(dataset_val), 
                              batch_size = 32)

## Task 7: Setting Up Optimizer and Scheduler

In [39]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [40]:
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, 
                  eps = 1e-8)

In [41]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = len(dataloader_train) * epochs
)

## Task 8: Defining our Performance Metrics

Accuracy metric approach originally used in accuracy function in [this tutorial](https://mccormickml.com/2019/07/22/BERT-fine-tuning/#41-bertforsequenceclassification).

In [42]:
import numpy as np

In [43]:
from sklearn.metrics import f1_score

In [44]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis = 1).flatten()
    labels_flat = labels.flatten()
    # weighted_f1_score (e.g. underrepresented classes are underweighted)
    return f1_score(preds_flat, labels_flat, average = 'weighted')

In [66]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

# Task 9: Creating our Training Loop

Approach adapted from an older version of HuggingFace's `run_glue.py` script. Accessible [here](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128).

In [46]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [47]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [48]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


In [50]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train,
                       desc='Epoch {:1d}'.format(epoch),
                       leave = False,
                       disable = False)
    
    for batch in progress_bar:
        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {
            'input_ids':      batch[0],
            'attention_mask': batch[1],
            'labels' :        batch[2]
        }
        
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_ (model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item())})
    
    torch.save(model.state_dict(), f'Models/BERT_ft_{epoch}.model')
    
    tqdm.write('\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    
    tqdm.write(f'Training loss {loss_train_avg}')
     
    val_loss, predictions, true_vals = evaluate(dataloader_val) 
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss {val_loss}')
    tqdm.write(f'F1 score (weighted): {val_f1}')


Epoch {epoch}
Training loss 0.47596282637160686
Validation loss 0.634927015219416
F1 score (weighted): 0.8707763362345563



Epoch {epoch}
Training loss 0.3293640828861426
Validation loss 0.602847264281341
F1 score (weighted): 0.8696982729927316



Epoch {epoch}
Training loss 0.22206214639942146
Validation loss 0.6578022390604019
F1 score (weighted): 0.9000460666959383



Epoch {epoch}
Training loss 0.14196826534674664
Validation loss 0.6922822722366878
F1 score (weighted): 0.8738672033556371



Epoch {epoch}
Training loss 0.09611124585378944
Validation loss 0.7152568591492516
F1 score (weighted): 0.8728376149661542



Epoch {epoch}
Training loss 0.06069342981236765
Validation loss 0.7773730371679578
F1 score (weighted): 0.864277294043012



Epoch {epoch}
Training loss 0.03384784920970791
Validation loss 0.7450711471693856
F1 score (weighted): 0.8780431078637356



Epoch {epoch}
Training loss 0.03198254298546263
Validation loss 0.7404736599751881
F1 score (weighted): 0.8827897003345433



Epoch {epoch}
Training loss 0.02374475921794314
Validation loss 0.7388718192066465
F1 score (weighted): 0.8860018344883815



Epoch {epoch}
Training loss 0.023245064994978113
Validation loss 0.7388718192066465
F1 score (weighted): 0.8860018344883815



## Task 10: Loading and Evaluating our Model

In [52]:
model.load_state_dict(torch.load('Models/BERT_ft_10.model', 
                                 map_location=torch.device('cuda')))

<All keys matched successfully>

In [54]:
_, predictions, true_vals = evaluate(dataloader_val)

In [67]:
accuracy_per_class(predictions, true_vals)

Class: happy
Accuracy: 165/171

Class: not-relevant
Accuracy: 19/32

Class: angry
Accuracy: 8/9

Class: disgust
Accuracy: 0/1

Class: sad
Accuracy: 2/5

Class: surprise
Accuracy: 2/5

